In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time

In [ ]:
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
train_images = (train_images / 255.0).astype(np.float32)
test_images = (test_images / 255.0).astype(np.float32)
train_labels = tf.keras.utils.to_categorical(train_labels)
test_labels = tf.keras.utils.to_categorical(test_labels)

In [ ]:
start_time = time.time()
interpreter = tf.lite.Interpreter(model_path="models/trained_model_non_concrete.tflite")
interpreter.allocate_tensors()
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
print("Input details:", input_details[0]['shape'])

In [ ]:
def evaluate_native (infer_fn):
    correct_predictions = 0
    true_labels = np.argmax(test_labels, axis=1)
    for i in range(len(test_images)):
        input_data = test_images[i].reshape(1, 28, 28) 
        output_data=infer_fn(x=input_data)
        predicted_label = np.argmax(output_data["output"])
        if predicted_label == true_labels[i]:
            correct_predictions += 1
    accuracy = correct_predictions / len(test_images)
    return accuracy

In [ ]:
initial_accuracy=evaluate_native(interpreter.get_signature_runner("infer"))
print(f"Initial Model Accuracy: {initial_accuracy:.4f}")

NUM_EPOCHS = 5
BATCH_SIZE = 32
epochs = np.arange(1, NUM_EPOCHS + 1, 1)
losses = np.zeros([NUM_EPOCHS])

train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_ds = train_ds.batch(BATCH_SIZE)

train_fn = interpreter.get_signature_runner("train")

for i in range(NUM_EPOCHS):
  for x,y in train_ds:
    result = train_fn(x=x, y=y)

  losses[i] = result['loss']
  print(f"Finished {i+1} epochs")
  print(f"  loss: {losses[i]:.3f}")
evaluate_native(interpreter.get_signature_runner("infer"))

In [ ]:
plt.plot(epochs, losses, label='Finetraining')
plt.ylim([0, max(plt.ylim())])
plt.xlabel('Epoch')
plt.ylabel('Loss [Cross Entropy]')
plt.legend()

In [ ]:
# save to ckpt-> save to tflite
save_fn = interpreter.get_signature_runner("save")
checkpoint_path = "models/trained_weights.ckpt"

save_fn(checkpoint_path=np.array(checkpoint_path, dtype=np.string_))
print(f"✅ Fine-tuned weights saved to {checkpoint_path}")

In [ ]:
# create model (must match original one)
model = tf.keras.Sequential([
   tf.keras.layers.Flatten(input_shape=(28, 28), name='flatten'),
    tf.keras.layers.Dense(128, activation='relu', name='dense_1'),
    tf.keras.layers.Dense(10, name='dense_2')
])
model.compile(optimizer='adam', loss=tf.keras.losses.CategoricalCrossentropy()) 

IMG_SIZE = 28
@tf.function(input_signature=[
    tf.TensorSpec([None, IMG_SIZE, IMG_SIZE], tf.float32),
    tf.TensorSpec([None, 10], tf.float32),
])
def train(x, y):
    with tf.GradientTape() as tape:
        prediction = model(x)
        loss = model.loss(y, prediction)
    gradients = tape.gradient(loss, model.trainable_variables)
    model.optimizer.apply_gradients(
        zip(gradients, model.trainable_variables))
    result = {"loss": loss}
    return result

@tf.function(input_signature=[
    tf.TensorSpec([None, IMG_SIZE, IMG_SIZE], tf.float32),
])
def infer(x):
    logits = model(x)
    probabilities = tf.nn.softmax(logits, axis=-1)
    return {
        "output": probabilities,
        "logits": logits
    }
@tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
def save(checkpoint_path):
    tensor_names = [weight.name for weight in model.weights]
    tensors_to_save = [weight.read_value() for weight in model.weights]
    tf.raw_ops.Save(
        filename=checkpoint_path, tensor_names=tensor_names,
        data=tensors_to_save, name='save')
    return {
        "checkpoint_path": checkpoint_path
    }

@tf.function(input_signature=[tf.TensorSpec(shape=[], dtype=tf.string)])
def restore(checkpoint_path):
    restored_tensors = {}
    for var in model.weights:
        restored = tf.raw_ops.Restore(
            file_pattern=checkpoint_path, tensor_name=var.name, dt=var.dtype,
            name='restore')
        var.assign(restored)
        restored_tensors[var.name] = restored
    return restored_tensors

signatures = {
    'train': train,
    'infer': infer,
    'save': save,
    'restore': restore
}

In [ ]:
#copy tensors into model variable
tensor_details = interpreter.get_tensor_details()
trainable_indices = [idx for idx,d in enumerate(tensor_details) if "variable" in d["name"].lower()]
weights = [interpreter.get_tensor(i) for i in trainable_indices]
model.layers[1].set_weights([weights[1], weights[0]]) 
model.layers[2].set_weights([weights[3], weights[2]])  

In [ ]:

SAVED_MODEL_DIR = "saved_model"

tf.saved_model.save(
    model,
    SAVED_MODEL_DIR,
    signatures=signatures
)
converter = tf.lite.TFLiteConverter.from_saved_model(SAVED_MODEL_DIR)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,  # Enable TensorFlow Lite ops.
    tf.lite.OpsSet.SELECT_TF_OPS  # Enable TensorFlow ops.
]
converter.experimental_enable_resource_variables = True

tflite_model = converter.convert()

# Save the TFLite model
tflite_model_path = "models/finetuned_model.tflite"
with open(tflite_model_path, "wb") as f:
    f.write(tflite_model)

print(f"✅ Fine-tuned TFLite model with signatures saved as: {tflite_model_path}")

In [ ]:
another_interpreter = tf.lite.Interpreter(model_content=tflite_model)
another_interpreter.allocate_tensors()
print("Model Signatures:", another_interpreter.get_signature_list())
infer_fn_new = another_interpreter.get_signature_runner("infer")
#optional as tflite contains the weights
# restore_fn_new = another_interpreter.get_signature_runner("restore")
# restore_fn_new(checkpoint_path=np.array(checkpoint_path, dtype=np.string_))

In [ ]:
new_accuracy= evaluate_native(infer_fn_new)
print(f"Final Model Accuracy: {new_accuracy:.4f}")
print(f"Model Improvement: {new_accuracy-initial_accuracy:.4f}")
end_time = time.time()
print(f"Total script execution time: {end_time - start_time:.2f} seconds")